In [5]:
pip install pymssql pandas matplotlib reportlab scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.2 MB/s eta 0:00:00


In [8]:
import os
import pymssql
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

# SQL Connection (pymssql)
# -----------------------------
SERVER = "smartfeedback-sql-signal.database.windows.net"
DB = "SmartFeedbackDB"
USER = "sqladmin"
PASSWORD = "Signal-dev"

conn = pymssql.connect(
    server=SERVER,
    user=USER,
    password=PASSWORD,
    database=DB
)

# -----------------------------
# reporting views
# -----------------------------
def load_public_impact(pract_id):
    q = f"SELECT * FROM vw_PractitionerPublicImpact WHERE PractitionerID='{pract_id}'"
    return pd.read_sql(q, conn)

def load_trends(pract_id):
    q = f"""
    SELECT * FROM vw_EmotionalStateTrends
    WHERE PractitionerID='{pract_id}'
    ORDER BY WeekNumber
    """
    return pd.read_sql(q, conn)

def load_sentiments(pract_id):
    q = f"SELECT * FROM vw_SentimentRollups WHERE PractitionerID='{pract_id}'"
    return pd.read_sql(q, conn)

def load_comments(pract_id):
    q = f"""
    SELECT Question AS comment, Sentiment, WeekNumber
    FROM ClientFeedback_Fact
    WHERE PractitionerID='{pract_id}'
      AND Question IS NOT NULL
      AND LTRIM(RTRIM(Question)) <> ''
    """
    return pd.read_sql(q, conn)


def top_themes(comments, top_n=5):
    if comments.empty:
        return []

    vectorizer = TfidfVectorizer(stop_words="english", max_features=2000)
    X = vectorizer.fit_transform(comments.astype(str))
    scores = X.sum(axis=0).A1
    vocab = vectorizer.get_feature_names_out()

    top_idx = scores.argsort()[::-1][:top_n]
    return [(vocab[i], round(scores[i], 3)) for i in top_idx]

# -----------------------------
#  creating Charts
# -----------------------------
def save_trend_chart(trends, out_path):
    plt.figure()
    plt.plot(trends["WeekNumber"], trends["AvgEmotionalChange"], marker="o")
    plt.title("Weekly Emotional Change Trend")
    plt.xlabel("Week")
    plt.ylabel("Avg Emotional Change")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

def save_sentiment_chart(sentiments, out_path):
    plt.figure()
    plt.bar(sentiments["Sentiment"], sentiments["FeedbackCount"])
    plt.title("Sentiment Breakdown")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

# -----------------------------
# PDF Generator
# -----------------------------
def generate_pdf(pract_id):

    impact = load_public_impact(pract_id)
    trends = load_trends(pract_id)
    sentiments = load_sentiments(pract_id)
    comments_df = load_comments(pract_id)

    if impact.empty:
        raise ValueError(f"No data found for PractitionerID={pract_id}")

    avg_impact = round(float(impact["AvgImpact"].iloc[0]), 2)
    positive_ratio = round(float(impact["PositiveRatio"].iloc[0]) * 100, 1)
    total = int(impact["TotalResponses"].iloc[0])

    themes = top_themes(comments_df["comment"], 5)

    trend_img = f"trend_{pract_id}.png"
    senti_img = f"sentiment_{pract_id}.png"

    if not trends.empty:
        save_trend_chart(trends, trend_img)

    if not sentiments.empty:
        save_sentiment_chart(sentiments, senti_img)

    pdf_name = f"SweatSignal_ImpactReport_{pract_id}.pdf"
    c = canvas.Canvas(pdf_name, pagesize=letter)
    width, height = letter

    # Header
    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 60, "SweatSignal Impact Report")

    c.setFont("Helvetica", 12)
    c.drawString(50, height - 85, f"Practitioner: {pract_id}")
    c.drawString(50, height - 105, f"Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}")

    # KPIs
    c.setFont("Helvetica-Bold", 13)
    c.drawString(50, height - 140, "Key Metrics")
    c.setFont("Helvetica", 12)
    c.drawString(50, height - 160, f"Average Emotional Change: {avg_impact}")
    c.drawString(50, height - 180, f"Positive Experience Ratio: {positive_ratio}%")
    c.drawString(50, height - 200, f"Total Feedback Responses: {total}")

    # Themes
    c.setFont("Helvetica-Bold", 13)
    c.drawString(50, height - 235, "Top 5 Themes Clients Mention")
    c.setFont("Helvetica", 12)
    y = height - 255
    if themes:
        for word, score in themes:
            c.drawString(60, y, f"- {word} (score={score})")
            y -= 18
    else:
        c.drawString(60, y, "No comments available.")

    # Charts
    y_chart = y - 20
    if os.path.exists(trend_img):
        c.setFont("Helvetica-Bold", 13)
        c.drawString(50, y_chart, "Emotional Trendline")
        c.drawImage(ImageReader(trend_img), 50, y_chart - 220, width=500, height=200)
        y_chart -= 250

    if os.path.exists(senti_img):
        c.setFont("Helvetica-Bold", 13)
        c.drawString(50, y_chart, "Sentiment Summary")
        c.drawImage(ImageReader(senti_img), 50, y_chart - 220, width=500, height=200)

    c.showPage()
    c.save()

    # cleanup
    for f in [trend_img, senti_img]:
        if os.path.exists(f):
            os.remove(f)

    return pdf_name


# -----------------------------
# Run
# -----------------------------
print(generate_pdf("P001"))


/tmp/ipython-input-557432691.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not test

SweatSignal_ImpactReport_P001.pdf


/tmp/ipython-input-557432691.py:131: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  c.drawString(50, height - 105, f"Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}")


In [9]:
print(generate_pdf("P001"))


/tmp/ipython-input-557432691.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, conn)
/tmp/ipython-input-557432691.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not test

SweatSignal_ImpactReport_P001.pdf


/tmp/ipython-input-557432691.py:131: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  c.drawString(50, height - 105, f"Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}")


In [10]:
import os
os.listdir()


['.config', 'SweatSignal_ImpactReport_P001.pdf', 'sample_data']

In [11]:
from google.colab import files
files.download("SweatSignal_ImpactReport_P001.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>